In [1]:
import pandas as pd
import json
import os
from parrot import Parrot
from tqdm import tqdm
from nltk import sent_tokenize
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/daniel/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [22]:
hc3_df = pd.read_json(path_or_buf="../../datasets/reddit_eli5.jsonl", lines=True)
hc3_df

,question,human_answers,chatgpt_answers,index
0,"Why is every book I hear about a "" NY Times # ...","[Basically there are many categories of "" Best...",[There are many different best seller lists th...,NaN
1,"If salt is so bad for cars , why do we use it ...",[salt is good for not dying in car crashes and...,[Salt is used on roads to help melt ice and sn...,NaN
2,Why do we still have SD TV channels when HD lo...,[The way it works is that old TV stations got ...,[There are a few reasons why we still have SD ...,NaN
3,Why has nobody assassinated Kim Jong - un He i...,[You ca n't just go around assassinating the l...,[It is generally not acceptable or ethical to ...,NaN
4,How was airplane technology able to advance so...,[Wanting to kill the shit out of Germans drive...,[After the Wright Brothers made the first powe...,NaN
...,...,...,...,...
17107,Why is the general Xbox One reveal feedback ne...,[A few items people do n't seem to like : 1 . ...,[It's difficult to say exactly why the general...,NaN
17108,If infrared is thermal why does n't my remote ...,"[It 's very , very low power . Around the orde...",[Infrared light is a type of light that we can...,NaN
17109,Why are cars all about their brake - horse pow...,[Brake horsepower has nothing to do with brake...,"[Cars are not just about how fast they can go,...",NaN
17110,"What does a CO^2 scrubber do , and how does it...",[A [ scrubber ] ( URL_0 ) is a engineering pro...,[A CO2 scrubber is a device that removes carbo...,NaN


In [3]:
parrot = Parrot()

In [19]:
def paraphrase(p):
    paraphrased = []
    s = sent_tokenize(p)
    for phrase in s:
        paraphrases = parrot.augment(input_phrase = phrase, 
                                     max_return_phrases = 5, 
                                     use_gpu = True,
                                     do_diverse=False,
                                     adequacy_threshold = 0.90, 
                                     fluency_threshold = 0.80
                                    )
        if paraphrases:
            paraphrased.append(paraphrases[0][0])
        else:
            # remove the last character (a period) and append the original phrase
            paraphrased.append(phrase[0:-1])
    return '. '.join(paraphrased)
# phrase = hc3_df.at[0, 'chatgpt_answers'][0]
# print(phrase+ "\n")
# print(paraphrase(phrase))

In [23]:
# add a new column to the df
hc3_df["paraphrased_parrot"] = None

n = len(hc3_df)

for i in tqdm(range(n),
               desc="Loading…",
               ascii=False):
    list_of_chatgpt_ans = hc3_df.at[i, 'chatgpt_answers']
    list_of_paraphrased_responses = []

    if len(list_of_chatgpt_ans)<=0:
        hc3_df.at[i, 'paraphrased_parrot'] = list_of_paraphrased_responses
        continue
    
    for chatgpt_ans in list_of_chatgpt_ans:
        paraphrased = paraphrase(chatgpt_ans)
        list_of_paraphrased_responses.append(paraphrased)
    hc3_df.at[i, 'paraphrased_parrot'] = list_of_paraphrased_responses

hc3_df.head()

Loading…: 100%|██████████████████████████████████████████████████████████████████████| 17112/17112 [30:11:42<00:00,  6.35s/it]


,question,human_answers,chatgpt_answers,index,paraphrased_parrot
0,"Why is every book I hear about a "" NY Times # ...","[Basically there are many categories of "" Best...",[There are many different best seller lists th...,NaN,[there are many different best seller lists pu...
1,"If salt is so bad for cars , why do we use it ...",[salt is good for not dying in car crashes and...,[Salt is used on roads to help melt ice and sn...,NaN,[salt is used on roads to help melt ice and sn...
2,Why do we still have SD TV channels when HD lo...,[The way it works is that old TV stations got ...,[There are a few reasons why we still have SD ...,NaN,[there are several reasons why we still have s...
3,Why has nobody assassinated Kim Jong - un He i...,[You ca n't just go around assassinating the l...,[It is generally not acceptable or ethical to ...,NaN,[it is generally not acceptable or ethical to ...
4,How was airplane technology able to advance so...,[Wanting to kill the shit out of Germans drive...,[After the Wright Brothers made the first powe...,NaN,[after wright made the first powered flight in...


In [24]:
hc3_df.to_json("paraphrased_parrot.jsonl", orient="records", lines=True)

ValueError: 'lines' keyword only valid when 'orient' is records